In [1]:
### In this notebook, we used py_imessage to retrieve and send messages.

from py_imessage import imessage

from time import sleep 

import os

import time

from imessage_reader import fetch_data


#Connect to openai API , Get api_key from their website.  Please don't use my key as 
#there are limites of query per minutes. If we're both using it, our programs might both fail.
#You can go to openai.com to get a key for free. 
import openai 

openai.api_key=os.getenv("OPENAI_API_KEY")

In [29]:

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

## Imessage Auto reply 

### This program works as following: 

- 1. During a given time (you can give the value in max_time. Unit is in seconds. For example, max_time=600 means this program will run for 10 mins then stop.

- 2. During this time, it will retrive messages from your iphone. when it detectes a new message coming from a given number (You can give it here by modify the value of phone_number ). It will give it to ChatGPT API to get a response. 

- 3. You can specify your desrived persona in the chatGPT prompt , however you want it. The assistant role will reply it for you. Tell the assitantnt to speak as you. 

- 4. The response generated by chatGPT will be sent to the phone number.  Meanwhile, this mssage will get appened in the conversation as the history of the chat. 

- 5. Each time, message sent and received will be appended to the conversation. The whole chat histroy during this session will be used in the new prompt. 


### Notice

ChatGPT has some randomness when it comes to generating answers. 90% time it is working well but 10% it will appear something as 'As an AI langurage.....', which throws away the converation.  Right now I can only hardcode them.  Please let me know if you have better ideas. 

In [ ]:
# SENT_FROM_HIM = 0 
# SENT_FROM_JACKY = 1
# SENDER_NUMBER_INDEX      = 0
# MESSAGE_INDEX            = 1
# DATE_INDEX               = 2
# MESSAGE_TYPE_INDEX       = 3
# YOUR_NUMBER_INDEX        = 4
# WHO_SENT_THIS_TEXT_INDEX = 5

###### CHANGE THOSE PARAMETERS BEFORE YOU START THIS PROGRAM!!######

# The number of person you're trying to chat with. It doesn't have to be an iphone user. 
phone_number = os.getenv("TARGET_PHONE_NUM")  

## Set up the max time this program will run.  One hour= 3600. 
max_time = 3600



## Get the inital count of the message from this person. 
fd = fetch_data.FetchData()
messages  = fd.get_messages()
received_messages=[]
for message in messages:
    if phone_number == message[0] and message[5]==0: ## Get the message from this phone number and it was sent by (insead of to them)
        received_messages.append(message)
inial_message=received_messages[-1][1]  
last_message_count =len(received_messages) # get the initial message count

print (f"there are {last_message_count} messages initially from this person")
print (f"Lastest message from this person before start the chatbot: {inial_message} ")



start_time = time.time()


## Write your prompt here. 
conversation =  [  
{'role':'system', 'content':'You are an assistant to help Jacky reply messages. \
Jacky is a data scientist and CTO and co-founder of a start up company.\
Jacky likes stand up comedy, skateboarding, travelling and gardening.\
Jacky like techlogy and start up ideas.\
You speak as you are Jacky.' 
}
] 
 

while time.time() - start_time < max_time:
   
    time.sleep(15)
#     fd = ir.fetch_data.FetchData()
    messages  = fd.get_messages()
    received_messages=[]
    for message in messages:
        if phone_number == message[0] and message[5]==0: 
            received_messages.append(message)
    last_message = received_messages[-1][1]            # Get the lastest message. 
        
    current_message_count = len(received_messages)
    if current_message_count > last_message_count:   #whenever there is a new message from the other person
        last_message_count = current_message_count
        print (f"New message: {last_message}")
        conversation.append({'role':'user', 'content':last_message}) #append this message to conversation history list.
        
        response=get_completion_from_messages(conversation, temperature=1)
        
       

    # Do some mannual adjustments.. 

        
        if 'AI language model' in response :
            response = response.replace('As an AI langurage model', "")
            
        if 'chatbot' in response :
            response = response.replace('As a chatbot', "")
            
        if 'virtual assistant' in response :
            response = response.replace('As a virtual assistant', "")
            
        if 'chatbot' in response :
            response = response.replace('As a chatbot', "")
            
#         if ':' in response:
#             response = response.split(":")[1]   
       
#             print(response) 
        
#         if 'How can I help you?' in response:
#             response = response.replace('How can I help you?', "")

        time.sleep(10)
        guid = imessage.send(phone_number, response)
  
        conversation.append({'role':'assistant', 'content':response})
    
        print (conversation)
     
    
    else: 
        print (f'No new messages')
        
print("While loop ended")


there are 7 messages initially from this person
Lastest message from this person before start the chatbot: Ttyl 
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
